## Dense Net

In [2]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.utils.data import DataLoader, ConcatDataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import pickle
import os
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Root directory for dataset
root_dir="images"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 224

# Number of training epochs
num_epochs = 40

# Learning rate for optimizers

lr = 1e-4

# Model hyperparameters
depth = 40  # or something like 100 
growth_rate = 12
drop_rate = 0.1
# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1


In [4]:
# Root directory for dataset
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = dset.ImageFolder(root=root_dir, transform=transform)
class_to_idx = dataset.class_to_idx  # e.g., {'fake': 0, 'real': 1}
real_label = class_to_idx['real']
fake_label = class_to_idx['fake']

# Separate indices
real_indices = [i for i, (_, label) in enumerate(dataset) if label == real_label]
fake_indices = [i for i, (_, label) in enumerate(dataset) if label == fake_label]

# Shuffle for randomness
random.shuffle(real_indices)
random.shuffle(fake_indices)

KeyboardInterrupt: 

Balancing Sets

In [11]:
# Define split sizes
real_total = len(real_indices)  # 30,000
fake_total = len(fake_indices)  # 90,000

# Let's use up to 30,000 real and 30,000 fake (to keep balance)
real_count = 30000
fake_count = 30000

real_indices = real_indices[:real_count]
fake_indices = fake_indices[:fake_count]

# Split each class into train/val/test (70/15/15)
def split_indices(indices):
    total = len(indices)
    train = int(0.7 * total)
    val = int(0.15 * total)
    return indices[:train], indices[train:train+val], indices[train+val:]

real_train, real_val, real_test = split_indices(real_indices)
fake_train, fake_val, fake_test = split_indices(fake_indices)

# Combine real + fake for each set
train_indices = real_train + fake_train
val_indices = real_val + fake_val
test_indices = real_test + fake_test

# Shuffle combined sets
random.shuffle(train_indices)
random.shuffle(val_indices)
random.shuffle(test_indices)

In [12]:
from torch.utils.data import Subset

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
from collections import Counter
labels = [label for _, label in dataset]
count = Counter(labels)
print("Real:", count[dataset.class_to_idx['real']])
print("Fake:", count[dataset.class_to_idx['fake']])

KeyboardInterrupt: 

In [ ]:
from densenet.models.densenet import DenseNet3

net = DenseNet3(depth=depth, num_classes=2, growth_rate=growth_rate, dropRate=drop_rate)
#net.cuda()
net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))


# Loss Functions and Optimizers

In [16]:
optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Validation Function

In [17]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import numpy as np

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            probs = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class '1' (fake)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # Convert to numpy arrays
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    # Metrics
    acc = 100 * np.mean(all_preds == all_labels)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    cm = confusion_matrix(all_labels, all_preds)

    print(f"📊 Accuracy:  {acc:.2f}%")
    print(f"🎯 F1 Score:  {f1:.4f}")
    print(f"📌 Precision: {precision:.4f}")
    print(f"📥 Recall:    {recall:.4f}")
    print(f"📈 ROC-AUC:   {auc:.4f}")
    print(f"🔲 Confusion Matrix:\n{cm}")

    avg_loss = running_loss / len(dataloader)
    return {
    "accuracy": acc,
    "loss": avg_loss,
    "f1": f1,
    "precision": precision,
    "recall": recall,
    "roc_auc": auc,
    "confusion_matrix": cm
}

# Training Function

In [18]:
from torch.utils.tensorboard import SummaryWriter
import datetime

# Set up TensorBoard writer
log_dir = f"runs/deepfake_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
writer = SummaryWriter(log_dir=log_dir)

2025-04-07 00:29:19.794145: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-07 00:29:19.911783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743982159.962721  159426 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743982159.975786  159426 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-07 00:29:20.095812: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [19]:
import torch
import torch.nn as nn
from tqdm import tqdm

def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10):
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]"):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        train_loss = running_loss / len(train_loader)

        # 💡 Validation + Metrics
        val_metrics = evaluate(model, val_loader, criterion, device)
        val_acc = val_metrics["accuracy"]
        val_loss = val_metrics["loss"]

        print(f"\nEpoch {epoch+1}/{num_epochs} Summary:")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val   Loss: {val_loss:.4f}, Val   Acc: {val_acc:.2f}%")

        # 🧠 Log to TensorBoard
        writer.add_scalar("Loss/train", train_loss, epoch)
        writer.add_scalar("Loss/val", val_loss, epoch)
        writer.add_scalar("Accuracy/train", train_acc, epoch)
        writer.add_scalar("Accuracy/val", val_acc, epoch)
        writer.add_scalar("F1/val", val_metrics["f1"], epoch)
        writer.add_scalar("Precision/val", val_metrics["precision"], epoch)
        writer.add_scalar("Recall/val", val_metrics["recall"], epoch)
        writer.add_scalar("ROC-AUC/val", val_metrics["roc_auc"], epoch)

        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print("Best model saved.")

    writer.close()
    print(f"\n✅ Training complete. Best validation accuracy: {best_acc:.2f}%")


# Testing Function

In [20]:
def test_model(model, test_loader, criterion, device):
    print("\n🧪 Evaluating on test set...")
    model.load_state_dict(torch.load("best_model.pth"))
    test_metrics = evaluate(model, test_loader, criterion, device)
    
    print("\n📌 Final Test Metrics:")
    print(f"Accuracy:  {test_metrics['accuracy']:.2f}%")
    print(f"F1 Score:  {test_metrics['f1']:.4f}")
    print(f"Precision: {test_metrics['precision']:.4f}")
    print(f"Recall:    {test_metrics['recall']:.4f}")
    print(f"ROC AUC:   {test_metrics['roc_auc']:.4f}")
    print("Confusion Matrix:\n", test_metrics['confusion_matrix'])

In [ ]:
# Train
train_model(net, train_loader, val_loader, criterion, optimizer, device, num_epochs=20)

# Test (after training)
test_model(net, test_loader, criterion, device)

NameError: name 'train_model' is not defined